In [3]:
import cv2
import numpy as np
import time
import math
from tensorflow.keras.models import load_model
from ultralytics import YOLO
import cvzone

# Load models
spoofing_model = YOLO("../l_version_1_300.pt")
face_recognition_model = load_model('Neo_Face_Recognition.keras')

# Setup camera
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Define face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

confidence_threshold = 0.6
classNames = ["fake", "real"]

prev_frame_time = 0

while True:
    new_frame_time = time.time()
    success, img = cap.read()
    if not success:
        break
    
    # First, check for real or spoof using YOLO model
    results = spoofing_model(img, stream=True, verbose=False)
    real_face_detected = False

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = [int(x) for x in box.xyxy[0]]
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            
            if conf > confidence_threshold and classNames[cls] == 'real':
                real_face_detected = True
                cvzone.cornerRect(img, (x1, y1, x2-x1, y2-y1), colorC=(0, 255, 0), colorR=(0, 255, 0))
                cvzone.putTextRect(img, f'{classNames[cls].upper()} {int(conf*100)}%',
                                   (max(0, x1), max(35, y1)), scale=2, thickness=4, colorR=(0, 255, 0), colorB=(0, 255, 0))
                break
        if real_face_detected:
            break

    # If a real face is detected, proceed with face recognition
    if real_face_detected:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            face = img[y:y+h, x:x+w]
            face = cv2.resize(face, (224, 224))
            face = np.expand_dims(face, axis=0) / 255.0

            prediction = face_recognition_model.predict(face)
            label = np.argmax(prediction, axis=1)

            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(img, f'Label: {label}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    # Calculate and display FPS
    fps = 1 / (new_frame_time - prev_frame_time)
    prev_frame_time = new_frame_time
    cv2.putText(img, f'FPS: {int(fps)}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Video", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
